## Intro


In [3]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
llama = Ollama(model="llama2")

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a poet, like Shakespeare."),
    ("user", "{input}")
])
outputparser = StrOutputParser()
chain = prompt | llama | outputparser

In [ ]:
chain.invoke({"input": "Why is the sky blue?"})

## Retrieval Chain


In [12]:
from langchain_community.document_loaders import WebBaseLoader  # Or wikipedia
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
embeddings = OllamaEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()

In [10]:
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [11]:
prompt = ChatPromptTemplate.from_template("""
                                          Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>

                                          Question: {question}
""")
document_chain = create_stuff_documents_chain(llama, prompt)

In [14]:
document_chain.invoke({
  "question": "how can langsmith help with testing?",
  "context": [Document(page_content="langsmith can let you viualize test results")]
})

'Based on the context provided, Langsmith can help with testing by allowing users to visualize test results. This suggests that Langsmith is a tool or platform that provides visualizations of test data, which can be useful for understanding and analyzing test results.'

we can instead find the most relevant documents by using a retriveal chain